In [ ]:
#Helpful information
# https://stable-baselines.readthedocs.io/en/master/guide/examples.html#using-callback-monitoring-training
# actual rom files must be named rom.nes and placed in their specific folders... my path is:
# {path on colossus}
# Scenario files also go here, these are what determines how we reward the agent

# To Do:
# See which of the models need special policies.
# The plan was to try various baseline modesl and compare them, (Have each use 1m timesteps?)
# try to auto-optimize their parameters and compare again, then
# if we had time try to put together one of our own and see how it compares to theirs.
# Haven't yet had good luck with any models thus far. (maybe because of the MlpPolicy?)

# Game List: SMB, Life Force, Megaman? Must be NES games.

In [1]:
import retro
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, Conv3D, Conv1D
from keras.optimizers import Adam
import os 

#mlp means Multilayer perceptron, and is probably the fastest but worst.
#CnnPolicies are for images only. MlpPolicies are made for other type of features (e.g. robot joints) 
#Dunno what this means exactly, but I copied it from the documentation.
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnPolicy, CnnLstmPolicy, CnnLnLstmPolicy
from stable_baselines.deepq.policies import DQNPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C, ACER, ACKTR, DDPG, DQN, GAIL, PPO2, SAC, TRPO
 #Documentation is here: https://stable-baselines.readthedocs.io/en/master/guide/examples.html#using-callback-monitoring-training

Using TensorFlow backend.


In [3]:
env = retro.make(game="SuperMarioBros-Nes")
#env = retro.make(game="LifeForce-Nes")
#env = gym.make('CartPole-v0')
env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run

In [ ]:
# Probably want to add a timer here that concludes after training,
# and also route output to some kind of file that we can read later.
model = A2C(MlpPolicy, env, verbose=1)
#model = PPO2(CnnPolicy, env, verbose=1)
#model = ACER(MlpPolicy, env, verbose=1)
#model = ACKTR(MlpPolicy, env, verbose=1)
#model = DDPG(MlpPolicy, env, verbose=1)
#model = DQN(DQNPolicy, env, verbose=1)
#model = GAIL(MlpPolicy, env, verbose=1)
#model = PPO2(MlpPolicy, env, verbose=1)
#model = SAC(MlpPolicy, env, verbose=1)
#model = TRPO(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=100000)
#model.save("savedModel")


In [ ]:
# If you want to just load a model instead of retraining... we can use this to train on colossus
model = A2C.load("A2C5mMlp")

In [ ]:
obs = env.reset()
for i in range(5000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [ ]:
# To find out where to put .nes files
print(retro.__file__)

In [ ]:
# To see what games can be ran, so long as we aquire the rom files.
retro.data.list_games()